# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_single_lon120.yml']
argv = ["-c", "cfg_testing.yml"]

cfg_file = argv[1]

# Create Neural Networks

Rasp et al.'s CBRAIN `fc_model` is used here for reference. As we are
doing a simpler version, I've opted to create our own, shorter function
for NN creation.

In [ ]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense


def dense_nn(input_shape, output_shape, hidden_layers, activation):
    """
    Creates a dense NN in base of the parameters received
    """
    model = Sequential()
    model.add(Input(shape=input_shape))

    for n_layer_nodes in hidden_layers:
        model.add(Dense(n_layer_nodes, activation=activation))

    model.add(Dense(output_shape))
    return model


def fc_model(
    input_shape,
    output_shape,
    hidden_layers,
    activation,
    conservation_layer=False,
    inp_sub=None,
    inp_div=None,
    norm_q=None,
):
    inp = Input(shape=(input_shape,))

    # First hidden layer
    x = Dense(hidden_layers[0])(inp)
    x = act_layer(activation)(x)

    # Remaining hidden layers
    for h in hidden_layers[1:]:
        x = Dense(h)(x)
        x = act_layer(activation)(x)

    if conservation_layer:
        x = SurRadLayer(inp_sub, inp_div, norm_q)([inp, x])
        x = MassConsLayer(inp_sub, inp_div, norm_q)([inp, x])
        out = EntConsLayer(inp_sub, inp_div, norm_q)([inp, x])

    else:
        out = Dense(output_shape)(x)

    return tf.keras.models.Model(inp, out)


In [ ]:
from pathlib import Path
from utils.constants import SPCAM_Vars, DATA_FOLDER, ANCIL_FILE
import utils.utils as utils

class Variable:
    """
    Object that stores a SPCAM variable and one specific level.
    
    Both the level in hPa and its index in ancillaries are stored.
    
    The main way to create these objects is to use `parse_var_name()`.
    
    Attributes
    ----------
    spcam_var : SPCAM_Vars
        SPCAM_Vars object corresponding to the variable.
    level : str
        Altitude in hPa.
    level_idx : int
        Level index as found in the ancillaries file.
    """

    def __init__(self, spcam_var, level_altitude, level_idx):
        self.var = spcam_var
        self.level = level_altitude
        self.level_idx = level_idx

    @staticmethod
    def parse_var_name(var_name):
        """
        Parses a string of variable and name to a Variable object
        
        Parameters
        ----------
        var_name : str
            String that represents a variable and a level, with format:
            "{variable name}-{altitude in hPa}"
        
        Returns
        -------
        Variable
            Variable object that contains the information referenced
            in the string
        """
        values = var_name.split("-")
        spcam_name = values[0]
        dict_spcam_vars = {v.name: v for v in SPCAM_Vars}
        spcam_var = dict_spcam_vars[spcam_name]

        if spcam_var.dimensions == 2:
            level_altitude = level_idx = None
        elif spcam_var.dimensions == 3:
            levels, _, _ = utils.read_ancilaries(Path(DATA_FOLDER, ANCIL_FILE))
            level_altitude = float(values[1])
            level_idx = utils.find_closest_value(levels, level_altitude)

        return Variable(spcam_var, level_altitude, level_idx)

    def __str__(self):
        if self.var.dimensions == 2:
            return f"{self.var.name}"
        elif self.var.dimensions == 3:
            return f"{self.var.name}-{self.level}"

    def __repr__(self):
        return repr(str(self))


In [ ]:
import numpy as np
import tensorflow as tf

# # NOTE: This will generate a lot of models, it may be better to put
# # them on different folders
# MODEL_FILENAME_PATTERN = "model-{variable}-a{pc_alpha}-t{threshold}.h5"

MODEL_FILENAME_PATTERN = "models/a{pc_alpha}-t{threshold}/{output}.h5" # Folders

class ModelDescription:
    """
    Object that stores a Keras model and metainformation about it.
    
    Attributes
    ----------
    output : Variable
        Output variable of the model.
    pc_alpha : str
        Meta information. PC alpha used to find the parents.
    threshold : str
        Meta information. Gridpoint threshold used to select the parents.
    parents : list(Variable)
        List of the variables (and variable level) that cause the output
        variable.
    hidden_layers : list(int)
        Description of the hidden dense layers of the model
        (default [32, 32, 32]).
    activation : Keras-compatible activation function
        Activation function used for the hidden dense layers
        (default "relu").
    model : Keras model
        Model created using the given information.
        See `_build_model()`.
    input_vars_dict:
    
    #TODO
    
    """
    
    def __init__(
        self,
        output,
        parents,
        model_type,
        pc_alpha,
        threshold,
        hidden_layers,
        activation,
    ):
        """
        Parameters
        ----------
        output : str
            Output variable of the model in string format. See Variable.
        parents : list(str)
            List of strings for the variables that cause the output variable.
            See Variable.
        model_type : str
            
        pc_alpha : str
            Meta information. PC alpha used to find the parents.
        threshold : str
            Meta information. Gridpoint threshold used to select the parents.
        hidden_layers : list(int)
            Description of the hidden dense layers of the model.
        activation : Keras-compatible activation function
            Activation function used for the hidden dense layers.
        """
        self.output = Variable.parse_var_name(output)
        self.parents = [Variable.parse_var_name(p) for p in parents]
        self.model_type = model_type
        self.pc_alpha = pc_alpha
        self.threshold = threshold
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.model = self._build_model()
        self.input_vars_dict = ModelDescription._build_vars_dict(self.parents)
        self.output_vars_dict = ModelDescription._build_vars_dict([self.output])

    def _build_model(self):
        """
        Build a Keras model with the given information.
        
        Some parameters are not configurable, taken from Rasp et al.
        """
        input_shape = len(parents)
        input_shape = (input_shape,)
        model = dense_nn(
            input_shape=input_shape,
            output_shape=1,  # Only one output per model
            hidden_layers=self.hidden_layers,
            activation=self.activation,
        )
        model.compile(
            optimizer = "adam", # From train.py (default)
            loss = "mse", # From 006_8col_pnas_exact.yml
            metrics = [tf.keras.losses.mse], # From train.py (default)
        )
        return model
    
    @staticmethod
    def _build_vars_dict(list_variables):
        """
        Convert the given list of Variable into a dictionary to be used
        on the data generator.
        
        Parameters
        ----------
        list_variables : list(Variable)
            List of variables to be converted to the dictionary format
            used by the data generator
        
        Returns
        -------
        vars_dict : dict{str : list(int)}
            Dictionary of the form {ds_name : list of levels}, where
            "ds_name" is the name of the variable as stored in the
            dataset, and "list of levels" a list containing the indices
            of the levels of that variable to use, or None for 2D
            variables.
        """
        vars_dict = dict()
        for variable in list_variables:
            ds_name = variable.var.ds_name # Name used in the dataset
            if variable.var.dimensions == 2:
                vars_dict[ds_name] = None
            elif variable.var.dimensions == 3:
                levels = vars_dict.get(ds_name, list())
                levels.append(variable.level_idx)
                vars_dict[ds_name] = levels
        return vars_dict
    
    def generate_filename(self):
        return MODEL_FILENAME_PATTERN.format(
            output = str(self.output),
            pc_alpha = self.pc_alpha,
            threshold = self.threshold
        )
    
    def save_model(self):
        self.model.save(self.generate_filename())

    def __str__(self):
        return f"{self.model_type}: {self.output}, a{self.pc_alpha}-t{self.threshold}"

    def __repr__(self):
        return repr(str(self))

    def __hash__(self):
        return hash(str(self))

    def __eq__(self, other):
        return str(self) == str(other)


## Read aggregated results and generate CausalSingleNN models

Currently, results are loaded from a previously created file. This was
made for testing purposes, and we may want to replace it for an analysis.

In that replacement, the idea would be to execute more or less the same
as aggregate_results up to the moment the file is currently saved, and
continue with that object (aggregated results) here.

In [ ]:
# TODO Move to configuration
HIDDEN_LAYERS = [32, 32, 32]  # Arbitrary, I choose them because it's a small NN
ACTIVATION = "relu"

In [ ]:
import utils.utils as utils
from pathlib import Path
example_aggregated = Path(
    "./aggregated_results",
    cfg_file.replace(".yml", ".obj")
)
# example_aggregated = "./aggregated_results/cfg_testing.obj"
aggregated_results = utils.load_results(example_aggregated)


In [ ]:
model_descriptions = list()
for output, pc_alpha_dict in aggregated_results.items():
    print(output)
    if len(pc_alpha_dict) == 0:  # May be empty
        # TODO How to approach this?
        print("Empty results")
        pass
    for pc_alpha, pc_alpha_results in pc_alpha_dict.items():
        var_names = np.array(pc_alpha_results["var_names"])
        for threshold, parent_idxs in pc_alpha_results["parents"].items():
            parents = var_names[parent_idxs]
            model_description = ModelDescription(
                output,
                parents,
                "CausalSingleNN",
                pc_alpha,
                threshold,
                hidden_layers=HIDDEN_LAYERS,
                activation=ACTIVATION
            )
            model_descriptions.append(model_description)


## Generate SingleNN

In [ ]:
from utils.constants import SPCAM_Vars
# TODO Only parents & children in configuration
output_list = [] # TODO Outputs and levels
parents = [] # TODO Parents and levels
for output in output_list:
    model_description = ModelDescription(
        output,
        parents,
        "CausalSingleNN",
        pc_alpha,
        threshold,
        hidden_layers=HIDDEN_LAYERS,
        activation=ACTIVATION
    )

# Training

## Data generator

In [ ]:
# TODO? Move this to configuration & constants

DATA_TYPE = "year"
DATA_TYPE = "month" # For testing

DATA_FOLDER = "/work/bd1083/b309162/preprocessed_data"
if DATA_TYPE == "year":
    TRAIN_FN    = "002_train_1_year.nc"
    VALID_FN    = "005_valid_1_year.nc"
elif DATA_TYPE == "month":
    TRAIN_FN    = "002_train_1_month.nc"
    VALID_FN    = "000_valid_1_month.nc"
# NORM_FN     = "000_norm.nc"
NORM_FN_PNAS = (
    "/pf/b/b309198/projects/causal_discovery/rasp-et-al/data/"
    "001_norm.nc"
) # From Tom https://github.com/tbeucler/CBRAIN-CAM/tree/master/notebooks/tbeucler_devlog/UW_DATA

INPUT_SUB = "mean" # From 006_8col_pnas_exact.yml
INPUT_DIV = "maxrs" # From 006_8col_pnas_exact.yml
OUT_SCALE_DICT_FN = ( # From 006_8col_pnas_exact.yml
    "/work/bd1179/b309198/causal_discovery/rasp-et-al/CBRAIN-CAM/"
    "nn_config/scale_dicts/002_pnas_scaling.pkl" 
) # From 006_8col_pnas_exact.yml
BATCH_SIZE = 1024


In [ ]:
from neural_networks.cbrain.utils import load_pickle
out_scale_dict = load_pickle(OUT_SCALE_DICT_FN)


In [ ]:
from neural_networks.cbrain.data_generator import DataGenerator
from pathlib import Path

def build_train_generator(input_vars_dict, output_vars_dict):
    train_gen = DataGenerator(
        data_fn          = Path(DATA_FOLDER, TRAIN_FN),
        input_vars_dict       = input_vars_dict,
        output_vars_dict      = output_vars_dict,
        # norm_fn          = Path(DATA_FOLDER, NORM_FN),
        norm_fn          = NORM_FN_PNAS,
        input_transform  = (INPUT_SUB, INPUT_DIV),
        output_transform = out_scale_dict,
        batch_size       = BATCH_SIZE,
        shuffle          = True, # This feature doesn't seem to work
    )
    return train_gen


nlat=64
nlon=128
ngeo = nlat * nlon

def build_valid_generator(input_vars_dict, output_vars_dict):
    valid_gen = DataGenerator(
            data_fn          = Path(DATA_FOLDER, VALID_FN),
            input_vars_dict       = input_vars_dict,
            output_vars_dict      = output_vars_dict,
            norm_fn          = NORM_FN_PNAS,
            input_transform  = (INPUT_SUB, INPUT_DIV),
            output_transform = out_scale_dict,
            batch_size       = ngeo,
            shuffle          = False,
            #xarray           = True,
    )
    return valid_gen


## Fit models

In [ ]:
"""
This cell is only for developing purposes, to not have to restart the full
Notebook every time one of these files was changed.

Can be safely removed once they are stable.
"""
from importlib import reload
import neural_networks.cbrain.data_generator
import neural_networks.cbrain.normalization
import neural_networks.cbrain.utils
import neural_networks.cbrain.learning_rate_schedule
reload(neural_networks.cbrain.data_generator)
reload(neural_networks.cbrain.utils)
reload(neural_networks.cbrain.normalization)
reload(neural_networks.cbrain.learning_rate_schedule)

Train all models in the model list and store the results.

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from neural_networks.cbrain.learning_rate_schedule import LRUpdate

for model_description in model_descriptions:
    print(model_description)
    input_vars_dict = model_description.input_vars_dict
    output_vars_dict = model_description.output_vars_dict
    
    # TODO Test that the data is being taken correctly
    with build_train_generator(
        input_vars_dict, output_vars_dict
    ) as train_gen, build_valid_generator(
        input_vars_dict, output_vars_dict
    ) as valid_gen:
        lrs = LearningRateScheduler(LRUpdate(
            init_lr = 0.001, # From train.py (default)
            step = 1, # From 006_8col_pnas_exact.yml
            divide = 5 # From train.py (default)
        ))
        model_description.model.fit(
            x = train_gen,
            validation_data = valid_gen,
            epochs = 4, # From 006_8col_pnas_exact.yml
            # epochs = 18, # TODO: Decide number of epochs, maybe use early stopper
            # verbose = 0, # Silent
            # verbose = 2, # Summary on epoch
            callbacks = [lrs]
        )
        model_description.save_model()
#         print(len(train_gen))
        break # For testing
